In [276]:
import pandas as pd
import numpy as np
import ydata_profiling as dp
import warnings
warnings.filterwarnings('ignore')

In [66]:
# load in data
data = pd.read_csv("./data/athlete_events.csv")
nations_data = pd.read_csv("./data/noc_regions.csv")

In [67]:
# Cleaning column names to lower case
data.columns = [x.lower() for x in data.columns]
print(data.columns)

Index(['id', 'name', 'sex', 'age', 'height', 'weight', 'team', 'noc', 'games',
       'year', 'season', 'city', 'sport', 'event', 'medal'],
      dtype='object')


In [68]:
# Initial look at the data
dp.ProfileReport(data)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## What do I want to look into?

### Exploration:

    How many total athletes?
    How many years?
    How many unique games?
    How many unique host cities?
    How many nations?

Overall best nations across all years, ordered by gold silver then bronze

- then split by winter then summer
  - then split by male and female

Best athletes from over the year

- then split by male and female

Highest performing age brackets

- then split by male and female


In [212]:
athletes = len(data['name'].unique())
print(f"Number of athletes competing in all the games? {athletes}")

start = data['year'].min()
print(f"Starting year? {start}")

games = len(data['games'].unique())
print(f"Number of unique games? {games}")

cities = len(data['city'].unique())
print(f"How many cities have held games? {cities}")

nations = list(data['noc'].unique())
print(f"Number of nations that have competed in the games? {len(nations)}")

Number of athletes competing in all the games? 134732
Starting year? 1896
Number of unique games? 51
How many cities have held games? 42
Number of nations that have competed in the games? 230


## Combined nations medals for all time (Summer & Winter) (Male & Female)

In [277]:
def get_nation_overall_medal_dataframe(medal: str, year: int, data: pd.DataFrame) -> pd.DataFrame:
    """A function for counting the given medal for each nation returning a dataframe"""
    yearly_data = data[data['year'] == year].drop_duplicates(subset=['event', 'medal'])
    medals = yearly_data[(data['medal'] == medal) & (data['year'] == year) ][['noc', 'medal']]
    medals_by_nation = medals.groupby('noc')[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

def update_total_nations_medal_list_for_medal_and_nation(medal: str, count: int, noc: str, table: dict) -> None:
    """Given a medal type, a count and country update the countries totals list of dictionaries"""
    for nation in table:
        if nation['noc'] == noc:
            nation[medal] += count


def update_total_nations_medals(df: pd.DataFrame, table: dict) -> None:

        for row in df.itertuples(): 
            try:
                update_total_nations_medal_list_for_medal_and_nation("Gold", row.Gold, row.Index, table)
            except:
                ""
            try:
                update_total_nations_medal_list_for_medal_and_nation("Silver", row.Silver, row.Index, table)
            except:
                ""
            try:
                update_total_nations_medal_list_for_medal_and_nation("Bronze", row.Bronze, row.Index, table)
            except:
                ""


def make_nations_list():
    nations_medal_list = []
    for nation in nations:
        nations_medal_list.append({"noc":nation, "Gold":0, "Silver": 0, "Bronze":0})
    return nations_medal_list


def sort_and_make_table_df(medals_list: list, sort_by: list):
    table_df = pd.DataFrame(medals_list)
    table_df = table_df.sort_values(by=sort_by, ascending=False)
    return table_df


medal_list = ['Gold', 'Silver', 'Bronze']
df_concat_list = []

year = 1896
years = []
while year < 2016:
    years.append(year)
    year += 2 

total_nations_medals = make_nations_list()

for year in years:
    for medal in medal_list:
        nations_year_medals  = get_nation_overall_medal_dataframe(medal, year, data)
        update_total_nations_medals(nations_year_medals, total_nations_medals)

total_nations_medals_df = sort_and_make_table_df(total_nations_medals, medal_list)

print(f"Top 10 achieving Nations of all time: ")
total_nations_medals_df.head(10)


Top 10 achieving Nations of all time: 


,noc,Gold,Silver,Bronze
3,USA,1078,858,724
27,URS,461,365,329
40,GER,298,333,304
87,GBR,259,293,273
8,FRA,254,261,292
14,ITA,246,213,224
0,CHN,212,164,134
79,SWE,196,206,231
75,GDR,192,163,155
18,RUS,183,166,162


## Get Overall nations medal table totals for all nations

In [279]:
def get_nation_seasonal_medal_dataframe(medal: str, year: int, data: pd.DataFrame, season: str) -> pd.DataFrame:
    """A function for counting the given medal for each nation returning a dataframe"""
    yearly_data = data[data['year'] == year].drop_duplicates(subset=['event', 'medal'])
    medals = yearly_data[(data['medal'] == medal) & (data['year'] == year) & (data['season'] == season) ][['noc', 'medal']]
    medals_by_nation = medals.groupby('noc')[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

# list of dictionaries to store the cumulative gold, silver, bronze for each nation
total_nations_summer_medals = make_nations_list()
total_nations_winter_medals = make_nations_list()

# for each year, and each medal, get each nations medals
for year in years:
    for medal in medal_list:
        
        nations_year_medals_summer  = get_nation_seasonal_medal_dataframe(medal, year, data, 'Summer')
        nations_year_medals_winter  = get_nation_seasonal_medal_dataframe(medal, year, data, 'Winter')  
        update_total_nations_medals(nations_year_medals_summer, total_nations_summer_medals)
        update_total_nations_medals(nations_year_medals_winter, total_nations_winter_medals)

# make dataframes out of the data
total_nations_summer_medals_df = sort_and_make_table_df(total_nations_summer_medals, medal_list)
total_nations_winter_medals_df = sort_and_make_table_df(total_nations_winter_medals,  medal_list)

print(f"\nTop 10 achieving Nations for Summer season of all time: \n")
print(total_nations_summer_medals_df.head(10))

print(f"\nTop 10 achieving Nations for Winter season of all time: \n")
print(total_nations_winter_medals_df.head(10))



Top 10 achieving Nations for Summer season of all time: 

    noc  Gold  Silver  Bronze
3   USA   982     760     641
27  URS   385     311     270
87  GBR   248     289     261
8   FRA   223     230     246
40  GER   212     249     246
14  ITA   210     179     181
0   CHN   200     142     115
29  HUN   168     149     159
75  GDR   153     127     120
79  SWE   146     166     177

Top 10 achieving Nations for Winter season of all time: 

     noc  Gold  Silver  Bronze
5    NOR   111     106      97
3    USA    96      98      83
40   GER    86      84      58
27   URS    76      54      59
41   CAN    60      55      52
130  AUT    59      76      81
79   SWE    50      40      54
72   SUI    50      40      47
18   RUS    49      40      35
4    FIN    41      62      55


## World rankings for all medals ever issued divided by Summer and Winter

### Includes team sports (1 medal issued for each)

In [280]:
def get_nation_overall_seasonal_medal_dataframe(medal: str, season: str) -> pd.DataFrame:
    """A function for counting the given medal for each nation returning a dataframe"""
    medals = data[(data['medal'] == medal) & (
        data['season'] == season)][['noc', 'medal']]
    medals_by_nation = medals.groupby('noc')[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation


def get_concatenated_dataframe_for_medals(season: str):
    """Function which takes a season and return the data for all medals"""
    df_concat_list_season = []
    for medal in medal_list:
        df_concat_list_season.append(
            get_nation_overall_seasonal_medal_dataframe(medal, season))

    nations_medals_season = pd.concat(
        df_concat_list_season, axis=1).reset_index()
    nations_medals_season = nations_medals_season.sort_values(
        by=medal_list, ascending=False)

    return nations_medals_season


summer_season_nations_medals = get_concatenated_dataframe_for_medals('Summer')
winter_season_nations_medals = get_concatenated_dataframe_for_medals('Winter')

print("\nFor every medal every earned (multiple for team sports)\n")

print("Top 10 achieving nations in the Olympics: Summer\n")
print(summer_season_nations_medals.head(10))

print("\nTop 10 achieving nations in the Olympics: Winter\n")
print(winter_season_nations_medals.head(10))


For every medal every earned (multiple for team sports)

Top 10 achieving nations in the Olympics: Summer

     noc    Gold  Silver  Bronze
101  USA  2472.0  1333.0  1197.0
99   URS   832.0   635.0   596.0
36   GBR   636.0   729.0   620.0
39   GER   592.0   538.0   649.0
51   ITA   518.0   474.0   454.0
34   FRA   465.0   575.0   587.0
44   HUN   432.0   328.0   363.0
87   SWE   354.0   396.0   358.0
4    AUS   342.0   452.0   510.0
37   GDR   339.0   277.0   227.0

Top 10 achieving nations in the Olympics: Winter

    noc   Gold  Silver  Bronze
5   CAN  305.0   199.0   107.0
35  URS  250.0    97.0    93.0
36  USA  166.0   308.0   161.0
17  GER  153.0   136.0    97.0
26  NOR  151.0   165.0   127.0
32  SWE  125.0   126.0   177.0
28  RUS   94.0    89.0    77.0
1   AUT   79.0    98.0   103.0
30  SUI   76.0    70.0   129.0
12  FIN   66.0   145.0   215.0


## Get top achieving nations by Male and Female of all time

In [96]:
def get_nation_gender_medal_dataframe(medal: str, sex: str) -> pd.DataFrame:
    """A function for counting the given medal for each nation returning a dataframe"""
    medals = data[(data['medal'] == medal) & (
        data['sex'] == sex)][['noc', 'medal']]
    medals_by_nation_gender = medals.groupby('noc')[['medal']].count()
    medals_by_nation_gender.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation_gender


def get_concatenated_dataframe_for_medals_for_gender(sex: str) -> pd.DataFrame:
    """Function for returning a concatenated dataframe for all medals for a gender"""
    df_concat_list_gender = []
    for medal in medal_list:
        df_concat_list_gender.append(
            get_nation_gender_medal_dataframe(medal, sex))

    nations_medals_by_gender = pd.concat(
        df_concat_list_gender, axis=1).reset_index()
    nations_medals_by_gender = nations_medals_by_gender.sort_values(
        by=medal_list, ascending=False)

    return nations_medals_by_gender


nations_medals_by_gender_male = get_concatenated_dataframe_for_medals_for_gender(
    'M')
nations_medals_by_gender_female = get_concatenated_dataframe_for_medals_for_gender(
    'F')

print("Top 10 achieving nations by gender: Male, of all time\n")
print(nations_medals_by_gender_male.head(10))

print("\nTop 10 achieving nations by gender: Female, of all time\n")
print(nations_medals_by_gender_female.head(10))

Top 10 achieving nations by gender: Male, of all time

    noc    Gold  Silver  Bronze
95  USA  1786.0  1107.0   939.0
93  URS   772.0   551.0   493.0
34  GBR   579.0   608.0   486.0
37  GER   533.0   452.0   505.0
48  ITA   514.0   452.0   452.0
32  FRA   446.0   523.0   580.0
82  SWE   434.0   431.0   451.0
41  HUN   363.0   237.0   268.0
13  CAN   305.0   285.0   242.0
64  NOR   301.0   290.0   212.0

Top 10 achieving nations by gender: Female, of all time

    noc   Gold  Silver  Bronze
72  USA  852.0   534.0   419.0
71  URS  310.0   181.0   196.0
30  GER  212.0   222.0   241.0
58  RUS  212.0   187.0   154.0
13  CHN  210.0   249.0   195.0
29  GDR  188.0   142.0   107.0
3   AUS  179.0   178.0   190.0
12  CAN  158.0   153.0   209.0
48  NED  142.0   148.0   142.0
56  ROU  121.0   104.0   130.0


## Get top achieving athletes of all time

In [108]:
# Best athletes of both categories of all time

def get_athlete_medal_dataframe(medal: str) -> pd.DataFrame:
    """A function for counting the {given} medal for each nation returning a dataframe"""
    medals = data[data['medal'] == medal][['name','noc','sex','medal']]
    medals_by_nation = medals.groupby(['name', 'noc', 'sex'])[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

medal_list = ['Gold', 'Silver', 'Bronze']
df_concat_list = []
for medal in medal_list:
    df_concat_list.append(get_athlete_medal_dataframe(medal))

athlete_medals = pd.concat(df_concat_list, axis=1).reset_index()
athlete_medals = athlete_medals.sort_values(by=medal_list, ascending=False)

print(f"The top 10 most successful Olympic athletes of all time:")
print(athlete_medals.head(10))


The top 10 most successful Olympic athletes of all time:
                                                 name  noc sex  Gold  Silver  \
6678                          Michael Fred Phelps, II  USA   M  23.0     3.0   
7888                      Raymond Clarence "Ray" Ewry  USA   M  10.0     NaN   
5632               Larysa Semenivna Latynina (Diriy-)  URS   F   9.0     5.0   
7436                             Paavo Johannes Nurmi  FIN   M   9.0     3.0   
6381                                Mark Andrew Spitz  USA   M   9.0     1.0   
2953                   Frederick Carlton "Carl" Lewis  USA   M   9.0     1.0   
7268                              Ole Einar Bjrndalen  NOR   M   8.0     4.0   
4529  Jennifer Elisabeth "Jenny" Thompson (-Cumpelik)  USA   F   8.0     3.0   
8437                                       Sawao Kato  JPN   M   8.0     3.0   
6533                   Matthew Nicholas "Matt" Biondi  USA   M   8.0     2.0   

      Bronze  
6678     2.0  
7888     NaN  
5632     4.0  
74

## Get highest peforming Male and Female athletes of all time

In [275]:
# Best athletes of both categories of all time

def get_athlete_medal_gender_dataframe(medal: str, sex: str) -> pd.DataFrame:
    """A function for counting the {given} medal for each nation returning a dataframe"""
    medals = data[ (data['medal'] == medal) & (data['sex'] == sex)][['name','noc','sex','medal']]
    medals_by_nation = medals.groupby(['name', 'noc', 'sex'])[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

def get_concatenated_dataframe_for_athlete_medals_for_gender(sex: str) -> pd.DataFrame:
    """Function for returning a concatenated dataframe for all medals for a athletes for a gender"""
    df_concat_list_gender = []
    for medal in medal_list:
        df_concat_list_gender.append(
            get_athlete_medal_gender_dataframe(medal, sex))

    athlete_medals_by_gender = pd.concat(
        df_concat_list_gender, axis=1).reset_index()
    athlete_medals_by_gender = athlete_medals_by_gender.sort_values(
        by=medal_list, ascending=False)

    return athlete_medals_by_gender

male_athletes_medals = get_concatenated_dataframe_for_athlete_medals_for_gender('M')
female_athletes_medals = get_concatenated_dataframe_for_athlete_medals_for_gender('F')


print(f"\nThe top 10 most successful Male Olympic athletes of all time:\n")
print(male_athletes_medals.head(10))

print(f"\nThe top 10 most successful Female Olympic athletes of all time:\n")
print(female_athletes_medals.head(10))


The top 10 most successful Male Olympic athletes of all time:

                                name  noc sex  Gold  Silver  Bronze
4950         Michael Fred Phelps, II  USA   M  23.0     3.0     2.0
5858     Raymond Clarence "Ray" Ewry  USA   M  10.0     NaN     NaN
5480            Paavo Johannes Nurmi  FIN   M   9.0     3.0     NaN
4733               Mark Andrew Spitz  USA   M   9.0     1.0     1.0
2224  Frederick Carlton "Carl" Lewis  USA   M   9.0     1.0     NaN
5362             Ole Einar Bjrndalen  NOR   M   8.0     4.0     1.0
6299                      Sawao Kato  JPN   M   8.0     3.0     1.0
4847  Matthew Nicholas "Matt" Biondi  USA   M   8.0     2.0     1.0
6946              Usain St. Leo Bolt  JAM   M   8.0     NaN     NaN
5272    Nikolay Yefimovich Andrianov  URS   M   7.0     5.0     3.0

The top 10 most successful Female Olympic athletes of all time:

                                                 name  noc sex  Gold  Silver  \
1311               Larysa Semenivna Latyni

## Get top achieving athletes for a given nation

In [274]:
# Get top athletes for a particular nation 

def get_nation_athlete_medal_dataframe(medal: str, noc: str) -> pd.DataFrame:
    """A function for counting the {given} medal for each nation returning a dataframe"""
    medals = data[(data['medal'] == medal) & (data['noc'] == noc)][['name','noc','sex','medal']]
    medals_by_nation = medals.groupby(['name', 'noc', 'sex'])[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation


medal_list = ['Gold', 'Silver', 'Bronze']
noc = 'GBR'

df_concat_list = []
for medal in medal_list:
    df_concat_list.append(get_nation_athlete_medal_dataframe(medal, noc))

athlete_medals = pd.concat(df_concat_list, axis=1).reset_index()
athlete_medals = athlete_medals.sort_values(by=medal_list, ascending=False)

print(f"\nThe top 10 most successful Olympic athletes of all time for {noc}:\n")
print(athlete_medals.head(10))


The top 10 most successful Olympic athletes of all time for GBR:

                                   name  noc sex  Gold  Silver  Bronze
99       Christopher Andrew "Chris" Hoy  GBR   M   6.0     1.0     NaN
291                 Jason Francis Kenny  GBR   M   6.0     1.0     NaN
66                 Bradley Marc Wiggins  GBR   M   5.0     1.0     2.0
493  Stephen Geoffrey "Steven" Redgrave  GBR   M   5.0     NaN     1.0
251                        Henry Taylor  GBR   M   4.0     1.0     3.0
74       Charles Benedict "Ben" Ainslie  GBR   M   4.0     1.0     NaN
361        Laura Rebecca Trott (-Kenny)  GBR   F   4.0     NaN     NaN
393               Matthew Clive Pinsent  GBR   M   4.0     NaN     NaN
402      Mohamed Muktar Jama "Mo" Farah  GBR   M   4.0     NaN     NaN
412  Paolo Francesco "Paul"  Radmilovic  GBR   M   4.0     NaN     NaN


## Top achieving athletes for a given nation and gender

In [285]:
# Get top athletes for a particular nation 

def get_nation_athlete_medal_gender_dataframe(medal: str, noc: str, sex: str) -> pd.DataFrame:
    """A function for counting the {given} medal for each nation returning a dataframe"""
    medals = data[(data['medal'] == medal) & (data['noc'] == noc) & (data['sex'] == sex)][['name','noc','sex','medal']]
    medals_by_nation = medals.groupby(['name', 'noc', 'sex'])[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

def get_concatenated_dataframe_for_athlete_medals_for_gender_nation(noc: str, sex: str) -> pd.DataFrame:
    """Function for returning a concatenated dataframe for all medals for a athletes for a gender"""
    df_concat_list_gender = []
    for medal in medal_list:
        df_concat_list_gender.append(
            get_nation_athlete_medal_gender_dataframe(medal, noc ,sex))

    athlete_medals_by_gender = pd.concat(
        df_concat_list_gender, axis=1).reset_index()
    athlete_medals_by_gender = athlete_medals_by_gender.sort_values(
        by=medal_list, ascending=False)

    return athlete_medals_by_gender

noc = 'USA'

top_male_nation_athletes = get_concatenated_dataframe_for_athlete_medals_for_gender_nation(noc,'M')
top_female_nation_athletes = get_concatenated_dataframe_for_athlete_medals_for_gender_nation(noc,'F')
print(f"\nThe top 10 most successful Male Olympic athletes of all time for {noc}:\n")
print(top_male_nation_athletes.head(10))
print(f"\nThe top 10 most successful Female Olympic athletes of all time for {noc}:\n")
print(top_female_nation_athletes.head(10))



The top 10 most successful Male Olympic athletes of all time for USA:

                                 name  noc sex  Gold  Silver  Bronze
890           Michael Fred Phelps, II  USA   M  23.0     3.0     2.0
1007      Raymond Clarence "Ray" Ewry  USA   M  10.0     NaN     NaN
832                 Mark Andrew Spitz  USA   M   9.0     1.0     1.0
413    Frederick Carlton "Carl" Lewis  USA   M   9.0     1.0     NaN
857    Matthew Nicholas "Matt" Biondi  USA   M   8.0     2.0     1.0
284   Donald Arthur "Don" Schollander  USA   M   7.0     1.0     NaN
1125               Ryan Steven Lochte  USA   M   6.0     3.0     3.0
122              Carl Townsend Osburn  USA   M   5.0     4.0     2.0
431              Gary Wayne Hall, Jr.  USA   M   5.0     3.0     2.0
844            Martin Joseph Sheridan  USA   M   5.0     3.0     1.0

The top 10 most successful Female Olympic athletes of all time for USA:

                                                name  noc sex  Gold  Silver  \
242  Jennifer El

## Medal Table for a given year

In [200]:
# Get nations medals by year

def get_medal_count_for_each_nation(medal: str, data: pd.DataFrame) -> pd.DataFrame:
    """A function for counting the given medal for each nation's atheletes returning a dataframe"""
    medals = data[ (data['medal'] == medal)][['year', 'noc', 'medal']]
    medals_by_nation = medals.groupby('noc')[['medal']].count()
    medals_by_nation.rename(columns={'medal': medal}, inplace=True)

    return medals_by_nation

def get_results_for_given_year(data: pd.DataFrame) -> pd.DataFrame:

    df_concat_list = []
    for medal in medal_list:
        df_concat_list.append(get_medal_count_for_each_nation(medal, data))

    nations_medals_for_year = pd.concat(df_concat_list, axis=1).reset_index()
    nations_medals_for_year = nations_medals_for_year.sort_values(by=medal_list, ascending=False)
    return nations_medals_for_year


def find_nations_medal_table_for_year(year:int) -> pd.DataFrame:
    
    data_for_year = data[(data['year'] == year) & ~(data['medal'].isnull())]
    data_for_year = data_for_year.drop_duplicates(subset=['event', 'medal'])
    medals = get_results_for_given_year(data_for_year)
    
    return medals

year = 2016
medal_table = find_nations_medal_table_for_year(year)

print(f"Top 10 achieving nations in the {year} games: ")
medal_table.head(10)


Top 10 achieving nations in the 2016 games: 


,noc,Gold,Silver,Bronze
56,USA,46.0,36.0,37.0
21,GBR,27.0,23.0,15.0
10,CHN,26.0,18.0,23.0
44,RUS,19.0,17.0,19.0
23,GER,17.0,10.0,11.0
32,JPN,12.0,8.0,19.0
20,FRA,10.0,18.0,13.0
35,KOR,9.0,3.0,6.0
29,ITA,8.0,12.0,8.0
2,AUS,8.0,11.0,10.0


In [273]:
# Given a (year) and (season) -> find the total for each nations medals.





In [169]:
drop_duplicates = data.drop_duplicates()
drop_duplicates[(drop_duplicates['noc'] == 'GBR') & (drop_duplicates['year'] == 2016)]['medal'].value_counts()

Gold      64
Silver    55
Bronze    26
Name: medal, dtype: int64

In [187]:
data.loc[(data['noc'] == 'ARG') & (data['year'] == 2016) & ~(data['medal'].isnull())]


,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
30600,15770,Manuel Brunet,M,30.0,179.0,79.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
33937,17489,Facundo Callioni,M,30.0,183.0,77.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
36011,18500,Cecilia Carranza Saroli,F,29.0,164.0,63.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Sailing,Sailing Mixed Multihull,Gold
52837,27132,Juan Martn del Potro,M,27.0,198.0,97.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Tennis,Tennis Men's Singles,Silver
79264,40252,Juan Ignacio Gilardi,M,34.0,186.0,91.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
101873,51517,Isidoro Ibarra,M,23.0,175.0,75.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
101876,51519,Pedro Ibarra,M,30.0,174.0,75.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
132007,66437,Santiago Ral Lange Roberti,M,54.0,184.0,73.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Sailing,Sailing Mixed Multihull,Gold
142316,71399,Juan Martn Lpez,M,31.0,178.0,74.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
151919,76242,Luca Masso,M,22.0,186.0,82.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Hockey,Hockey Men's Hockey,Gold
